# Convert Supervisely json to txt labels
All the labeling work was done in Supervise.ly.  Their json format needs to be converted to KITTI txt in the format:

```
kitti_formatted
  ├── images
  │   ├── IMAGE_0011.jpg
  │   ├── ......
  │   └── IMAGE_2697.jpg
  └── labels
      ├── IMAGE_0011.txt
      ├── ......
      └── IMAGE_2697.txt
```

The format is described here:  https://github.com/NVIDIA/DIGITS/tree/master/digits/extensions/data/objectDetection

We will need to: 
* Enumerate our Object Classes (Turn text in number 1..n)
* Convert the json to KITTI format
  * I think
* Convert to tfrecords (This splits train/val)
* Retrain


In [4]:
!pwd

/Users/abbyblancett/wpilib/2020/NanoVision5607


In [23]:
import supervisely_lib as sly  # Supervisely Python SDK
import json                    # Add Python JSON module for pretty-printing.
import csv
# Load the project meta-data.
# This does NOT load the images or their labeling data in memory.
project = sly.Project('/Users/abbyblancett/wpilib/2020/2020team5607', sly.OpenMode.READ)

With an opened project, we can get some basic information: its name, location on disk, member datasets, the total number of images. We can also iterate over the datasets, and go over the images in each dataset.

In [24]:
print(project.meta)

ProjectMeta:
Object Classes
+--------------------+-----------+----------------+
|        Name        |   Shape   |     Color      |
+--------------------+-----------+----------------+
|    Blue Bottom     | Rectangle | [252, 200, 90] |
|        hole        | Rectangle | [13, 160, 158] |
|  Generator Switch  |  Polygon  | [188, 85, 16]  |
|     Blue Upper     | Rectangle | [91, 152, 251] |
| Green Color Wheel  |  Polygon  | [42, 153, 178] |
|     Red Upper      | Rectangle |  [149, 9, 89]  |
|      Red Hole      | Rectangle |  [208, 2, 27]  |
|  Red Color Wheel   |  Polygon  | [242, 121, 84] |
|     Red Lower      | Rectangle | [145, 13, 46]  |
|  Blue Color Wheel  |  Polygon  | [16, 11, 147]  |
| Yellow Color Wheel |  Polygon  | [201, 220, 40] |
|     Power_Cell     | Rectangle | [248, 231, 28] |
+--------------------+-----------+----------------+
Tags
+---------+------------+-----------------+
|   Name  | Value type | Possible values |
+---------+------------+-----------------+
| Inva

In [31]:
# Print basic project metadata.
print("Project name: ", project.name)
print("Project directory: ", project.directory)
print("Total images: ", project.total_items)
print("Dataset names: ", project.datasets.keys())
print("\n")

# What datasets and images are there and where are they on disk?
for dataset in project:
    print("Dataset: ", dataset.name)
    
    # A dataset item is a pair of an image and its annotation.
    # The annotation contains all the labeling information about
    # the image - segmentation masks, objects bounding boxes etc.
    # We will look at annotations in detail shortly.
    with open('/tmp/images.csv', 'w', newline='') as csvfile:
        imagewriter = csv.writer(csvfile, delimiter=',',
                                quotechar=' ', quoting=csv.QUOTE_MINIMAL)
        
        for item_name in dataset:
            print(item_name)
            img_path = dataset.get_img_path(item_name)
            ann_path = dataset.get_ann_path(item_name)
            print("  image: ", img_path)
            print("  ann:   ", ann_path)
            imagewriter.writerow([img_path, ann_path, 'other stuff'])
            ann = sly.Annotation.load_json_file(ann_path, project.meta)

            print('\tLoaded annotation has {} labels and {} image tags.'.format(len(ann.labels), len(ann.img_tags)))
            print('\tLabel class names: ' + (', '.join(label.obj_class.name for label in ann.labels)))
            print('\tImage tags: ' + (', '.join(tag.get_compact_str() for tag in ann.img_tags)))
            for label in ann.labels:
                print('\tLabel class: {}; type: {}; label area: {}'.format(
                        label.obj_class.name,
                        label.geometry.geometry_name(),
                        label.geometry.area))
                bbox = label.geometry.to_bbox()
                print('\tLabel bounding box: [{}, {}, {}, {}]'.format(
                    bbox.top, bbox.left, bbox.bottom, bbox.right))
#    for tag in label.tags:
#        print('Tag: ' + tag.get_compact_str())
print()

Project name:  2020team5607
Project directory:  /Users/abbyblancett/wpilib/2020/2020team5607
Total images:  4638
Dataset names:  ['filmingDay1Video', 'filmingDay2Video', 'filmingDay1Images']


Dataset:  filmingDay1Video
frame_02855.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/img/frame_02855.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/ann/frame_02855.png.json
	Loaded annotation has 0 labels and 1 image tags.
	Label class names: 
	Image tags: Invalid
frame_09900.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/img/frame_09900.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/ann/frame_09900.png.json
	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_01390.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/img/frame_01390.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/ann/frame_013

	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_04955.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/img/frame_04955.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/ann/frame_04955.png.json
	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_10920.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/img/frame_10920.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/ann/frame_10920.png.json
	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_11380.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/img/frame_11380.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/ann/frame_11380.png.json
	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_12845.png
  image:  /Users/abbyblancett

	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_11185.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/img/frame_11185.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/ann/frame_11185.png.json
	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_07095.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/img/frame_07095.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/ann/frame_07095.png.json
	Loaded annotation has 0 labels and 1 image tags.
	Label class names: 
	Image tags: Valid
frame_03595.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/img/frame_03595.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/ann/frame_03595.png.json
	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_09735.png
  image:  /Users/abbybla

	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_00510.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/img/frame_00510.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/ann/frame_00510.png.json
	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_11435.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/img/frame_11435.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/ann/frame_11435.png.json
	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_05440.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/img/frame_05440.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/ann/frame_05440.png.json
	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_01140.png
  image:  /Users/abbyblancett

	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_12990.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/img/frame_12990.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/ann/frame_12990.png.json
	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_05220.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/img/frame_05220.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/ann/frame_05220.png.json
	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_11255.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/img/frame_11255.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Video/ann/frame_11255.png.json
	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_01720.png
  image:  /Users/abbyblancett

...

	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_02145.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay2Video/img/frame_02145.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay2Video/ann/frame_02145.png.json
	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_06445.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay2Video/img/frame_06445.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay2Video/ann/frame_06445.png.json
	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_02810.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay2Video/img/frame_02810.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay2Video/ann/frame_02810.png.json
	Loaded annotation has 0 labels and 1 image tags.
	Label class names: 
	Image tags: Valid
frame_00975.png
  image:  /Users/abbybla

	Loaded annotation has 0 labels and 1 image tags.
	Label class names: 
	Image tags: Valid
frame_08550.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay2Video/img/frame_08550.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay2Video/ann/frame_08550.png.json
	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_05060.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay2Video/img/frame_05060.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay2Video/ann/frame_05060.png.json
	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_01560.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay2Video/img/frame_01560.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay2Video/ann/frame_01560.png.json
	Loaded annotation has 6 labels and 1 image tags.
	Label class names: Power_Cell, Power_Cell, Power_Cell, Power_Cell, Power_C

	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_00095.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay2Video/img/frame_00095.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay2Video/ann/frame_00095.png.json
	Loaded annotation has 8 labels and 1 image tags.
	Label class names: Power_Cell, Power_Cell, Power_Cell, Power_Cell, Power_Cell, Power_Cell, Power_Cell, Power_Cell
	Image tags: Valid
	Label class: Power_Cell; type: rectangle; label area: 625.0
	Label bounding box: [119, 201, 143, 225]
	Label class: Power_Cell; type: rectangle; label area: 650.0
	Label bounding box: [111, 231, 136, 255]
	Label class: Power_Cell; type: rectangle; label area: 528.0
	Label bounding box: [103, 260, 126, 281]
	Label class: Power_Cell; type: rectangle; label area: 700.0
	Label bounding box: [102, 371, 126, 398]
	Label class: Power_Cell; type: rectangle; label area: 460.0
	Label bounding box: [84, 418, 103, 440]
	Label class

	Loaded annotation has 1 labels and 0 image tags.
	Label class names: Red Lower
	Image tags: 
	Label class: Red Lower; type: rectangle; label area: 38766.0
	Label bounding box: [230, 211, 411, 423]
frame_09110.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay2Video/img/frame_09110.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay2Video/ann/frame_09110.png.json
	Loaded annotation has 0 labels and 0 image tags.
	Label class names: 
	Image tags: 
frame_02045.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay2Video/img/frame_02045.png
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay2Video/ann/frame_02045.png.json
	Loaded annotation has 1 labels and 1 image tags.
	Label class names: hole
	Image tags: Valid
	Label class: hole; type: rectangle; label area: 3080.0
	Label bounding box: [3, 351, 58, 405]
frame_02910.png
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay2Video/img/frame_02910.png
  ann:    

	Loaded annotation has 5 labels and 1 image tags.
	Label class names: Power_Cell, Power_Cell, Power_Cell, Power_Cell, Power_Cell
	Image tags: Valid
	Label class: Power_Cell; type: rectangle; label area: 391.0
	Label bounding box: [284, 454, 306, 470]
	Label class: Power_Cell; type: rectangle; label area: 342.0
	Label bounding box: [286, 413, 303, 431]
	Label class: Power_Cell; type: rectangle; label area: 240.0
	Label bounding box: [283, 167, 298, 181]
	Label class: Power_Cell; type: rectangle; label area: 306.0
	Label bounding box: [284, 135, 301, 151]
	Label class: Power_Cell; type: rectangle; label area: 210.0
	Label bounding box: [267, 26, 281, 39]
DSC02130.jpg
  image:  /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Images/img/DSC02130.jpg
  ann:    /Users/abbyblancett/wpilib/2020/2020team5607/filmingDay1Images/ann/DSC02130.jpg.json
	Loaded annotation has 0 labels and 1 image tags.
	Label class names: 
	Image tags: Invalid
DSC02124.jpg
  image:  /Users/abbyblancett/wpilib